# Comparing CART and MIO methods to create decision trees

Import packages needed for CART and MIO methods

In [14]:
import tree as miptree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from ucimlrepo import fetch_ucirepo
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
import pydot
from sklearn import tree

Define function to convert data into binary:

In [15]:
def binary_features(array):
    output = array.copy()
    for i in range(len(array)):
        if 0 < array.num[i]:
            output.num[i] = 1
    return output

Set arguments:

In [16]:
timelimit = 600
seed = 42
train_ratio = 0.5
val_ratio = 0.25
test_ratio = 0.25
max_depth=3
min_samples_split=2

Import and manipulate data:

In [21]:
# Import data
heart_disease = fetch_ucirepo(id=45)
x = heart_disease.data.features
y = heart_disease.data.targets
# Drop coloumns with NaN values
x = x.drop(labels=["ca","thal"], axis=1)
# Convert to numpy array
x = x.to_numpy()
y = binary_features(y)
y = y.values.flatten()
# Take only 75 samples
x = x[0:74]
y = y[0:74]

Split data into train, test, and validation sets:

In [22]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=1-train_ratio, random_state=seed)

# MIO

Create tree using miptree:

In [23]:
octree = miptree.optimalDecisionTreeClassifier(max_depth=max_depth, min_samples_split=0, alpha=0.01, timelimit=timelimit)
octree.fit(x_train, y_train)

Training data include 37 instances, 11 features.
Set parameter TimeLimit to value 600
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[x86])

CPU model: Intel(R) Core(TM) i5-1038NG7 CPU @ 2.00GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1313 rows, 443 columns and 13054 nonzeros
Model fingerprint: 0xf4005749
Variable types: 39 continuous, 404 integer (404 binary)
Coefficient statistics:
  Matrix range     [3e-02, 4e+01]
  Objective range  [1e-02, 5e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]

User MIP start produced solution with objective 0.107619 (0.03s)
Loaded user MIP start with objective 0.107619

Presolve removed 8 rows and 0 columns
Presolve time: 0.02s
Presolved: 1305 rows, 443 columns, 12154 nonzeros
Variable types: 7 continuous, 436 integer (404 binary)

Root relaxation: objective 0.000000e+00, 286 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Obj

Find accuracy of MIO for train data:

In [24]:
y_train_pred = octree.predict(x_train)
accuracy_score(y_train, y_train_pred)

1.0

Find accuracy of MIO for test data:

In [25]:
y_test_pred = octree.predict(x_test)
accuracy_score(y_test, y_test_pred)

0.6756756756756757

# CART

Create tree using CART:

In [26]:
tree = DecisionTreeClassifier(max_depth=max_depth, min_samples_split=min_samples_split, ccp_alpha = 0.01)
tree.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.01, max_depth=3)

Find accuracy of CART for train data:

In [27]:
y_train_pred = tree.predict(x_train)
accuracy_score(y_train, y_train_pred)

0.972972972972973

Find accuracy of CART for test data:

In [28]:
y_test_pred = tree.predict(x_test)
accuracy_score(y_test, y_test_pred)

0.7027027027027027